In [1]:
#import dependencies
import pandas as pd
from datetime import datetime
import re
import nltk
nltk.download('wordnet')
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\flurp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
#import data from all tweets
df = pd.read_csv("../Raw_Data/Hydrated/all_geo_tweets.csv", dtype={"text": "string"})

In [13]:
df.head()


,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,...,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count,user_time_zone,user_urls,user_verified
0,"-74.0064,40.7142",Fri Apr 10 04:26:17 +0000 2020,follow alone apocalypse corona covid nyc manha...,NaN,https://www.instagram.com/p/B-yWtu4HcFO/?igshi...,0,1248467312014798848,NaN,NaN,NaN,...,1615,1246,22,333 Park Avenue South NYC USA,"Emil William Chynn, MD, FACS, MBA",EmilChynn,6372,NaN,http://www.ParkAvenueLASEK.com,False
1,"36.81666667,-1.28333333",Fri Apr 10 08:41:56 +0000 2020,thenewtoronto3,NaN,https://www.instagram.com/p/B-y0UVNBdW6/?igshi...,0,1248531648351801345,NaN,NaN,NaN,...,1925,43,2,Worldwide,Honorable Dice,honorabledice,679,NaN,http://donadostone.fanlink.to/sunday-morning,False
2,"35,31",Fri Apr 10 09:52:03 +0000 2020,CovidIsrael,NaN,https://corona-scanner.com/country/israel http...,0,1248549294723997696,neilellis,1.248549e+18,14292054.0,...,524,257,60,UK,Neil Ellis,neilellis,34720,NaN,NaN,False
3,"3.36494044,6.57778852",Fri Apr 10 08:51:57 +0000 2020,NaN,NaN,https://www.instagram.com/p/B-y1dzwpfwmwr3pZgx...,1,1248534172324560896,NaN,NaN,NaN,...,182,826,0,"Lagos, Nigeria",JUSTICE,bushman4u2me,255,NaN,NaN,False
4,NaN,Fri Apr 10 04:58:09 +0000 2020,LockdownSA corona quarantine lockedin rightnow...,NaN,https://www.instagram.com/p/B-yatfHlXCg/?igshi...,0,1248475333705076739,NaN,NaN,NaN,...,822,1720,20,Cyberspace,Tech Star,thegearsh,9544,NaN,http://www.thegearsh.co.za,False


In [14]:
# Create two lists for the loop results to be placed
lat = []
lon = []

# For each row in a varible,
for row in df['coordinates']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat.append(row.split(',')[1])
        # Split the row by comma and append
        # everything after the comma to lon
        lon.append(row.split(',')[0])
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
df['latitude'] = lat
df['longitude'] = lon
df

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,...,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count,user_time_zone,user_urls,user_verified,latitude,longitude
0,"-74.0064,40.7142",Fri Apr 10 04:26:17 +0000 2020,follow alone apocalypse corona covid nyc manha...,NaN,https://www.instagram.com/p/B-yWtu4HcFO/?igshi...,0,1248467312014798848,NaN,NaN,NaN,...,22,333 Park Avenue South NYC USA,"Emil William Chynn, MD, FACS, MBA",EmilChynn,6372,NaN,http://www.ParkAvenueLASEK.com,False,40.7142,-74.0064
1,"36.81666667,-1.28333333",Fri Apr 10 08:41:56 +0000 2020,thenewtoronto3,NaN,https://www.instagram.com/p/B-y0UVNBdW6/?igshi...,0,1248531648351801345,NaN,NaN,NaN,...,2,Worldwide,Honorable Dice,honorabledice,679,NaN,http://donadostone.fanlink.to/sunday-morning,False,-1.28333333,36.81666667
2,"35,31",Fri Apr 10 09:52:03 +0000 2020,CovidIsrael,NaN,https://corona-scanner.com/country/israel http...,0,1248549294723997696,neilellis,1.248549e+18,14292054.0,...,60,UK,Neil Ellis,neilellis,34720,NaN,NaN,False,31,35
3,"3.36494044,6.57778852",Fri Apr 10 08:51:57 +0000 2020,NaN,NaN,https://www.instagram.com/p/B-y1dzwpfwmwr3pZgx...,1,1248534172324560896,NaN,NaN,NaN,...,0,"Lagos, Nigeria",JUSTICE,bushman4u2me,255,NaN,NaN,False,6.57778852,3.36494044
4,NaN,Fri Apr 10 04:58:09 +0000 2020,LockdownSA corona quarantine lockedin rightnow...,NaN,https://www.instagram.com/p/B-yatfHlXCg/?igshi...,0,1248475333705076739,NaN,NaN,NaN,...,20,Cyberspace,Tech Star,thegearsh,9544,NaN,http://www.thegearsh.co.za,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252713,"-86.0113,40.0456",Thu Sep 10 03:11:00 +0000 2020,NaN,NaN,https://www.instagram.com/p/CE8MEihlMbR/?igshi...,1,1303893712053956609,NaN,NaN,NaN,...,60,NaN,Charlie Hopper,CharlieHopper,10027,NaN,http://SellingEating.com,False,40.0456,-86.0113
252714,"-117.868184,33.601926",Thu Sep 10 03:24:49 +0000 2020,NaN,NaN,https://www.instagram.com/p/CE8NpuPhdFp/?igshi...,0,1303897186875432960,NaN,NaN,NaN,...,57,"Irvine, CA",Marla Stone,ideal_organizer,5195,NaN,http://www.i-deal-lifestyle.com,False,33.601926,-117.868184
252715,"77.20609412,28.51820604",Thu Sep 10 03:36:35 +0000 2020,escape travel luxury life,NaN,https://www.instagram.com/p/CE8O_nilqDwM5kg1Wu...,0,1303900147097759747,cntraveller,NaN,36065771.0,...,35,Delhi,vishalvirsingh,vishalvirsingh,47261,NaN,NaN,False,28.51820604,77.20609412
252716,"77.219672,28.631747",Thu Sep 10 04:00:29 +0000 2020,finance metal compressedair aircompressor mone...,NaN,https://www.instagram.com/p/CE8Ru0WDxBY/?igshi...,0,1303906163306749952,NaN,NaN,NaN,...,1,"Noida, India",neha Nagar,nehaangar,77,NaN,http://taxationhelp.in,False,28.631747,77.219672


In [15]:
df = df.drop(columns=['media','urls', 'user_urls', 'user_statuses_count', "user_time_zone", 
                      "user_verified","retweet_id", "retweet_screen_name", "in_reply_to_screen_name", 
                      "in_reply_to_status_id", "in_reply_to_user_id","lang", "possibly_sensitive", 
                      "user_default_profile_image", "user_description", "user_favourites_count",
                      "user_followers_count", "user_listed_count", "user_screen_name.1", "source" ])

In [16]:
df.count()

coordinates           252596
created_at            252718
hashtags              139191
favorite_count        252718
id                    252718
place                 252098
retweet_count         252718
text                  252718
tweet_url             252718
user_created_at       252718
user_screen_name      252718
user_friends_count    252718
user_location         236557
user_name             252718
latitude              252596
longitude             252596
dtype: int64

In [18]:
#create dataframe with coordinates no text
geo_df = df[["latitude", "longitude", "hashtags", "place", "id"]]
geo_df

,latitude,longitude,hashtags,place,id
0,40.7142,-74.0064,follow alone apocalypse corona covid nyc manha...,"Manhattan, NY",1248467312014798848
1,-1.28333333,36.81666667,thenewtoronto3,"Nairobi, Kenya",1248531648351801345
2,31,35,CovidIsrael,Israel,1248549294723997696
3,6.57778852,3.36494044,NaN,"Ikeja, Nigeria",1248534172324560896
4,NaN,NaN,LockdownSA corona quarantine lockedin rightnow...,NaN,1248475333705076739
...,...,...,...,...,...
252713,40.0456,-86.0113,NaN,"Noblesville, IN",1303893712053956609
252714,33.601926,-117.868184,NaN,"Newport Beach, CA",1303897186875432960
252715,28.51820604,77.20609412,escape travel luxury life,"New Delhi, India",1303900147097759747
252716,28.631747,77.219672,finance metal compressedair aircompressor mone...,"New Delhi, India",1303906163306749952


In [21]:
#create dataframe with text
text_df = df[["latitude", "longitude", "hashtags", "place", "id", "text"]]
text_df = text_df.dropna(subset=['latitude', 'longitude', 'hashtags'])
text_df.reset_index(drop=True)
text_df.astype({'id': 'object'}).dtypes
text_df

,latitude,longitude,hashtags,place,id,text
0,40.7142,-74.0064,follow alone apocalypse corona covid nyc manha...,"Manhattan, NY",1248467312014798848,"At night these days in Manhattan, one can sit ..."
1,-1.28333333,36.81666667,thenewtoronto3,"Nairobi, Kenya",1248531648351801345,If you don’t want catch bodies just like Coro...
2,31,35,CovidIsrael,Israel,1248549294723997696,"🇮🇱 Israel 🔮: 10📆 183💀 🔮: 68📆 31,267🤧 438💀 🦠x..."
5,56.2639,9.5018,CovidDenmark,"Silkeborg, Danmark",1248511969298087936,"🇩🇰 Denmark 🔮: 10📆 518💀 🔮: 440📆 545,427🤧 7,636..."
6,5.1213,-1.2714,JAY9ICEGH,"Cape Coast, Ghana",1248474767679574016,Is a new day. Good morning ghana Stay home sta...
...,...,...,...,...,...,...
252710,-37.8156,144.9661,melbourne melbournemusic covid19 lockdown stay...,"Melbourne, Victoria",1303910614457372675,"What lockdown? 🧔keep smiling guys, we can do t..."
252711,-38.15729913,145.16968766,wearamask bumface crackhead,"Melbourne, Victoria",1303893314278748160,I had an idea for a mask with a bum on it........
252715,28.51820604,77.20609412,escape travel luxury life,"New Delhi, India",1303900147097759747,"@cntraveller ,the only magazine that I still r..."
252716,28.631747,77.219672,finance metal compressedair aircompressor mone...,"New Delhi, India",1303906163306749952,"What do you think guys? Before corona also ,..."


In [22]:
final_df = geo_df.dropna(subset=['latitude', 'longitude', 'hashtags', 'id'])
final_df.reset_index(drop=True, inplace=True)
final_df.astype({'id': 'object'}).dtypes
final_df

,latitude,longitude,hashtags,place,id
0,40.7142,-74.0064,follow alone apocalypse corona covid nyc manha...,"Manhattan, NY",1248467312014798848
1,-1.28333333,36.81666667,thenewtoronto3,"Nairobi, Kenya",1248531648351801345
2,31,35,CovidIsrael,Israel,1248549294723997696
3,56.2639,9.5018,CovidDenmark,"Silkeborg, Danmark",1248511969298087936
4,5.1213,-1.2714,JAY9ICEGH,"Cape Coast, Ghana",1248474767679574016
...,...,...,...,...,...
139116,-37.8156,144.9661,melbourne melbournemusic covid19 lockdown stay...,"Melbourne, Victoria",1303910614457372675
139117,-38.15729913,145.16968766,wearamask bumface crackhead,"Melbourne, Victoria",1303893314278748160
139118,28.51820604,77.20609412,escape travel luxury life,"New Delhi, India",1303900147097759747
139119,28.631747,77.219672,finance metal compressedair aircompressor mone...,"New Delhi, India",1303906163306749952


In [23]:
#create CSV
final_df.to_csv("../Raw_Data/Hydrated/cleaned_geo.csv")
text_df.to_csv("../Raw_Data/Hydrated/cleaned_geo_text.csv")

In [ ]:
final = final_df["place"].value_counts().head(6000)

In [ ]:
(final_df["place"] == "Jamaica").value_counts()